In [ ]:
import pandas as pd
import pickle


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error
import gdown
import wandb

In [ ]:
# download
data_path: str = "data/data_raw"
train_data_path = f"{data_path}/green_taxi_data_01.csv"
test_data_path = f"{data_path}/green_taxi_data_02.csv"
gdown.cached_download(
    f"https://drive.google.com/uc?id=1P7h3ni1T-QY_ECw4EWPSZ8tdzzlgDgcU", train_data_path, quiet=False
)
gdown.cached_download(
    f"https://drive.google.com/uc?id=1eFdBVdQlYtxxfkqW_c-C9A5Kqsb06O80", test_data_path, quiet=False
)

In [10]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

In [11]:
df_train = read_dataframe("data/data_raw/green_taxi_data_01.csv")
df_val = read_dataframe("data/data_raw/green_taxi_data_02.csv")


/tmp/ipykernel_159627/4283963972.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)


In [ ]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)


root_mean_squared_error(y_val, y_pred)


7.758715204764688